In [3]:
import numpy as np
import random
from collections import defaultdict

class TicTacToe:
    def __init__(self, board_size=3, win_length=3):
        self.board_size = board_size
        self.win_length = win_length  # number of markers in a row needed to win
        self.reset()

    def reset(self):
        self.board = np.zeros((self.board_size, self.board_size), dtype=int)
        self.current_player = 1  # 1 for agent, -1 for opponent
        self.done = False
        return self.get_state()

    def get_state(self):
        # Represent state as a tuple of rows so that it is hashable.
        return tuple(map(tuple, self.board))

    def available_actions(self):
        # Return list of indices (i, j) for empty cells.
        return list(zip(*np.where(self.board == 0)))

    def step(self, action):
        i, j = action
        if self.board[i, j] != 0:
            # Illegal move, can penalize heavily
            return self.get_state(), -10, True  # end the game on illegal move

        # Make the move
        self.board[i, j] = self.current_player
        
        # Check for win or draw
        if self.check_win(i, j):
            reward = 1 if self.current_player == 1 else -1
            self.done = True
        elif len(self.available_actions()) == 0:
            reward = 0  # draw
            self.done = True
        else:
            reward = 0
            self.current_player *= -1  # switch player

        return self.get_state(), reward, self.done

    def check_win(self, row, col):
        # Check all directions: horizontal, vertical, and two diagonals.
        directions = [(0,1), (1,0), (1,1), (1,-1)]
        for dr, dc in directions:
            count = 1  # count the current cell
            # Check one direction
            r, c = row + dr, col + dc
            while 0 <= r < self.board_size and 0 <= c < self.board_size and self.board[r, c] == self.current_player:
                count += 1
                r += dr
                c += dc
            # Check the opposite direction
            r, c = row - dr, col - dc
            while 0 <= r < self.board_size and 0 <= c < self.board_size and self.board[r, c] == self.current_player:
                count += 1
                r -= dr
                c -= dc
            if count >= self.win_length:
                return True
        return False

# Q-learning agent
class QLearningAgent:
    def __init__(self, alpha=0.1, gamma=0.9, epsilon=1.0, epsilon_decay=0.995, min_epsilon=0.01):
        self.Q = defaultdict(float)  # Q-table, key: (state, action), value: Q-value
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.min_epsilon = min_epsilon

    def choose_action(self, state, available_actions):
        if random.random() < self.epsilon:
            return random.choice(available_actions)
        q_values = [self.Q[(state, action)] for action in available_actions]
        max_q = max(q_values)
        # If multiple actions have the same Q-value, choose one at random.
        best_actions = [action for action, q in zip(available_actions, q_values) if q == max_q]
        return random.choice(best_actions)

    def learn(self, state, action, reward, next_state, done, available_actions_next):
        current_q = self.Q[(state, action)]
        if done:
            target = reward
        else:
            next_q = max([self.Q[(next_state, a)] for a in available_actions_next], default=0)
            target = reward + self.gamma * next_q
        self.Q[(state, action)] = current_q + self.alpha * (target - current_q)

    def update_epsilon(self):
        self.epsilon = max(self.epsilon * self.epsilon_decay, self.min_epsilon)

# Training loop
def train(agent, env, episodes=5000):
    for episode in range(episodes):
        state = env.reset()
        while True:
            available_actions = env.available_actions()
            action = agent.choose_action(state, available_actions)
            next_state, reward, done = env.step(action)
            agent.learn(state, action, reward, next_state, done, env.available_actions())
            state = next_state
            if done:
                break
        agent.update_epsilon()
        if (episode+1) % 500 == 0:
            print(f"Episode {episode+1}/{episodes} - Epsilon: {agent.epsilon:.3f}")

if __name__ == '__main__':
    board_size = 4  # You can change this to any size you like.
    win_length = 4  # Change accordingly if you want a different win condition.
    env = TicTacToe(board_size=board_size, win_length=win_length)
    agent = QLearningAgent()

    train(agent, env, episodes=5000)

    # Test a single game after training
    state = env.reset()
    while True:
        print(np.array(state))
        available_actions = env.available_actions()
        action = agent.choose_action(state, available_actions)
        state, reward, done = env.step(action)
        if done:
            print(np.array(state))
            print("Reward:", reward)
            break


Episode 500/5000 - Epsilon: 0.082
Episode 1000/5000 - Epsilon: 0.010
Episode 1500/5000 - Epsilon: 0.010
Episode 2000/5000 - Epsilon: 0.010
Episode 2500/5000 - Epsilon: 0.010
Episode 3000/5000 - Epsilon: 0.010
Episode 3500/5000 - Epsilon: 0.010
Episode 4000/5000 - Epsilon: 0.010
Episode 4500/5000 - Epsilon: 0.010
Episode 5000/5000 - Epsilon: 0.010
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
[[0 1 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
[[ 0  1  0  0]
 [ 0  0 -1  0]
 [ 0  0  0  0]
 [ 0  0  0  0]]
[[ 0  1  0  0]
 [ 0  0 -1  0]
 [ 0  0  0  1]
 [ 0  0  0  0]]
[[ 0  1  0 -1]
 [ 0  0 -1  0]
 [ 0  0  0  1]
 [ 0  0  0  0]]
[[ 0  1  0 -1]
 [ 0  0 -1  0]
 [ 0  0  0  1]
 [ 0  0  0  1]]
[[ 0  1  0 -1]
 [ 0  0 -1  0]
 [ 0  0  0  1]
 [ 0 -1  0  1]]
[[ 0  1  1 -1]
 [ 0  0 -1  0]
 [ 0  0  0  1]
 [ 0 -1  0  1]]
[[ 0  1  1 -1]
 [ 0  0 -1 -1]
 [ 0  0  0  1]
 [ 0 -1  0  1]]
[[ 0  1  1 -1]
 [ 0  1 -1 -1]
 [ 0  0  0  1]
 [ 0 -1  0  1]]
[[ 0  1  1 -1]
 [ 0  1 -1 -1]
 [-1  0  0  1]
 [ 0 -1  0  1]]
[[ 1  1  1 -